In [ ]:
import os
from PIL import Image, ImageFile
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor

ImageFile.LOAD_TRUNCATED_IMAGES = True

def process_image(img_path, input_dir, output_dir, max_size_mb=5, quality=100):
    try:
        rel_path = os.path.relpath(img_path, input_dir)
        output_path = Path(output_dir, rel_path).with_suffix(".jpg")  
        output_path.parent.mkdir(parents=True, exist_ok=True)

        with Image.open(img_path) as img:
            orig_size = os.path.getsize(img_path)
            max_size_bytes = max_size_mb * 1024 * 1024
            if img.mode not in ('RGB', 'L'):
                img = img.convert('RGB')
            scale = (max_size_bytes / orig_size) ** 0.5 if orig_size > max_size_bytes else 1.0
            new_size = (max(1, int(img.width * scale)), max(1, int(img.height * scale)))
            img.thumbnail(new_size, Image.Resampling.LANCZOS)
            current_quality = quality
            while current_quality > 5:
                img.save(output_path, 'JPEG', quality=current_quality, optimize=True)
                if os.path.getsize(output_path) <= max_size_bytes:
                    break
                current_quality -= 5

            print(f"✅ {rel_path}: {orig_size/1024/1024:.2f}MB → {os.path.getsize(output_path)/1024/1024:.2f}MB")

    except Exception as e:
        print(f"❌ Error processing {img_path}: {e}")

def resize_images(input_dir, output_dir, max_size_mb=5, quality=85, num_threads=4):
    input_dir, output_dir = Path(input_dir), Path(output_dir)
    
    # Find all image files
    all_images = [os.path.join(root, file) 
                  for root, _, files in os.walk(input_dir) 
                  for file in files if file.lower().endswith(('jpg', 'jpeg', 'png', 'bmp', 'tif', 'tiff'))]

    print(f"📂 Found {len(all_images)} images. Processing...")

    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        executor.map(lambda img: process_image(img, input_dir, output_dir, max_size_mb, quality), all_images)

    print("Processing complete!")

# Example usage
if __name__ == '__main__':
    resize_images(
        input_dir=r'D:\Major Project\Dataset\Resized, Augmented\Segmented_Augmented',
        output_dir=r'D:\Major Project\Dataset\Resized, Augmented\New\New Dataset',
        max_size_mb=5,
        quality=85,
        num_threads=8 
    )
